# **Setup and Library Imports**

### **Installing Required Packages**

In [34]:
! pip install --quiet transformers
! pip install --quiet torch
! pip install --quiet datasets
! pip install --quiet scikit-learn
! pip install --quiet matplotlib
! pip install --quiet ipywidgets
! pip install --quiet seaborn

### **Importing Libraries**

In [35]:
import os

# Hugging Face libraries for training and transformer models
from transformers import pipeline

# Evaluation metrics and utilities
from sklearn.metrics import confusion_matrix, classification_report

# Loading datasets for training and evaluation
from datasets import load_dataset

# Data manipulation and display utilities
import pandas as pd
import numpy as np
from tabulate import tabulate
from collections import Counter

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


### **Logging into Hugging Face Hub**

In [36]:
from huggingface_hub import notebook_login

# Execute the login function to access the Hugging Face account
# notebook_login()

### **Defining Model and Dataset Paths**

In [ ]:
models_path = [
    "SodaXII/dinov2-base_rice-disease-02_111824",
    "SodaXII/vit-base-patch16-224_rice-disease-02_111724",
]

dataset_path = "cvmil/rice-disease-02"

# Define the output directory for the test results
output_dir = "./testing_output/"

### **Load Dataset and Extract Labels**

Load the dataset from huggingface and extract the class labels from the training data.

In [39]:
dataset = load_dataset(dataset_path, split="test")
dataset = dataset.train_test_split(test_size=0.5, seed=42)["test"]
labels = dataset.features["label"].names

Generate and display a table showing class distribution of the test set.

In [ ]:
data = {
    "Label": labels,
    "Test": Counter(dataset["label"]).values(),
}

print(tabulate(data, headers="keys", tablefmt="psql"))

# **Test Pipieline**

Function to display and save confusion matrix

In [ ]:
def plot_and_save_confusion_matrix(y_true, y_pred, labels, model_name):
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Create a heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    
    # Save the confusion matrix
    os.makedirs(f"{output_dir}/{model_name}", exist_ok=True)
    plt.savefig(f"{output_dir}/{model_name}/confusion_matrix.png")
    plt.show()
    plt.close()

Function to save classification report

In [42]:
def save_classification_report(y_true, y_pred, labels, model_name):
    report = classification_report(y_true, y_pred, target_names=labels)
    print(report)
    
    # Save the report to a text file
    os.makedirs(f"{output_dir}/{model_name}", exist_ok=True)
    with open(f"{output_dir}/{model_name}/classification_report.txt", "w") as f:
        f.write(report)

### **Model Evaluation**

This section evaluates the model on the dataset by running inference on each image and comparing the predicted label with the actual label.

This section computes and displays the confusion matrix and classification report, providing insights into the model's performance on the dataset.

In [ ]:
for model_path in models_path:
    model_name = model_path.split("/")[-1]
    classifier = pipeline("image-classification", model=model_path, device='cpu')
    
    predictions = [
        classifier(img["image"])[0] for img in tqdm(dataset, desc=f"Processing {model_name}")
    ]
    y_true = dataset["label"]
    y_pred = [labels.index(pred["label"]) for pred in predictions]
    
    plot_and_save_confusion_matrix(y_true, y_pred, labels, model_name)
    save_classification_report(y_true, y_pred, labels, model_name)

print("Evaluation completed!")